### **LLM API 활용해보기**

**[앤트로픽의 Claude 2.1 모델 API 호출 코드]**

In [ ]:
import anthropic

anthropic.Anthropic(
    api_key="YOUR_API_KEY").messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, world"}
    ]
)

**[오픈AI의 GPT-3.5 Turbo 모델 API 호출 코드]**

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="YOUR_API_KEY")
client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": "Who won the world series in 2020?"
        }
    ]
)

**[랭체인을 활용한 앤트로픽 Claude 2.1 모델 API 호출 코드]**

In [ ]:
from langchain_anthropic import ChatAnthropic 
chat = ChatAnthropic(
    model_name="claude-3-haiku-20240307",
    anthropic_api_key="YOUR_API_KEY"
)
chat.invoke("안녕~ 너를 소개해줄래?")

**[랭체인을 활용한 오픈AI GPT-3.5 Turbo 모델 API 호출 코드]**

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name='gpt-4o-mini',
    openai_api_key="YOUR_API_KEY"
)
chat.invoke("안녕~ 너를 소개해줄래?")

### **프롬프트 템플릿에 대해 알아보기**

**[ChatPromptTemplate]**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
	#SystemMessage: 유용한 챗봇이라는 역할과 이름을 부여
        ("system", "You are a helpful AI bot. Your name is {name}."), 
    #HumanMessage와 AIMessage: 서로 안부를 묻고 답하는 대화 히스토리 주입
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
    #HumanMessage로 사용자가 입력한 프롬프트를 전달
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

In [ ]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
       "You are a helpful assistant that re-writes the user's text to sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

### **LLM API의 다양한 기능 활용해보기**

**[LLM API의 Temperature 이애하기]**

In [ ]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#OPENAI API키 저장
os.environ["OPENAI_API_KEY"]="YOUR_OPENAI_API_KEY"

#Temperature=0
chatgpt_temp0_1 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)
chatgpt_temp0_2 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)

#Temperature=1
chatgpt_temp1_1 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 1)
chatgpt_temp1_2 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 1)

model_list = [chatgpt_temp0_1, chatgpt_temp0_2, chatgpt_temp1_1, chatgpt_temp1_2]

for i in model_list:
    answer = i.invoke("왜 파이썬이 가장 인기있는 프로그래밍 언어인지 한 문장으로 설명해줘", max_tokens = 128)
    print("-"*100)
    print(">>>",answer.content)

**[ChatGPT처럼 답변 스트리밍하기]**

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)
for chunk in chat.stream("달에 관한 시를 써줘"):
    print(chunk.content, end="", flush=True)

**[답변 캐싱하기]**

In [ ]:
from langchain.globals import set_llm_cache #캐시메모리 라이브러리 호출
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)

**첫 질문-응답 시간 측정**

In [ ]:
%%time 
#셀 실행 시간 측정
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache()) #캐시메모리 설정

chat.invoke("일반상대성 이론을 한마디로 설명해줘")

**두번째 질문-응답 시간 측정**

In [ ]:
%%time
#같은 질문 전달
chat.invoke("일반상대성 이론을 한마디로 설명해줘")

### **실습**

**[이번 장에서 배운 것 실습해보기] - 스트리밍되는 AI스터디 플래너 챗봇 만들기**

In [ ]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

#GPT-3.5 모델 호출
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)

#ChatPromptTemplate 통해 스터디 플래너 역할 부여 및 사용자 프롬프트 매개변수화
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "당신은 공부 계획을 세워주는 스터디 플래너 머신입니다."
                "사용자의 공부 주제를 입력 받으면, 이를 학습하기 위한 공부 계획을 작성합니다."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

#앞서 설정한 프롬프트 템플릿에 HumanMessage로 문장 전달
messages = chat_template.format_messages(text="Large Language Model에 대해서 공부하고 싶어요.")

#stream 함수를 통해 답변 스트리밍
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)